<a href="https://colab.research.google.com/github/Art-phys/Lesson_HF_LR/blob/main/Lesson_HF_RL_Unit8_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Блок 8, Часть 2: Углубленное обучение с подкреплением. Использование Sample Factory для воспроизведения Doom из пикселей

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit9/thumbnail2.png" alt="Thumbnail" width="70%"/>

В этом блокноте мы узнаем, как обучить глубокую нейронную сеть собирать объекты в 3D-среде на основе игры Doom, видео результирующей политики показано ниже. Мы разрабатываем эту политику, используя [Sample Factory](https://www.samplefactory.dev/), асинхронная реализация алгоритма PPO.

Пожалуйста, обратите внимание на следующие моменты:

*   [Sample Factory](https://www.samplefactory.dev/) это продвинутый фреймворк RL и **работает только в Linux и Mac** (не в Windows).

*  Фреймворк лучше всего работает на ** машине с графическим процессором и большим количеством процессорных ядер **, где он может достигать скорости 100 тыс. взаимодействий в секунду. Ресурсы, доступные в стандартном ноутбуке Colab **, ограничивают производительность этой библиотеки**. Таким образом, скорость в этом параметре ** не отражает реальную производительность **.
* Контрольные показатели для Sample Factory доступны в ряде настроек, ознакомьтесь с [примерами](https://github.com/alex-petrenko/sample-factory/tree/master/sf_examples) если вы хотите узнать больше.


In [1]:
from IPython.display import HTML

HTML('''<video width="640" height="480" controls>
  <source src="https://huggingface.co/edbeeching/doom_health_gathering_supreme_3333/resolve/main/replay.mp4" 
  type="video/mp4">Your browser does not support the video tag.</video>'''
)

Чтобы подтвердить это практическое руководство для [процесса сертификации](https://huggingface.co/deep-rl-course/en/unit0/introduction#certification-process), вам нужно обучить модель:

- `doom_health_gathering_supreme` получив результат >= 5.

Чтобы найти свой результат, перейдите к [leaderboard](https://huggingface.co/spaces/huggingface-projects/Deep-Reinforcement-Learning-Leaderboard) и найдите свою модель, **the result = mean_reward - std of reward**

Прежде чем приступить к обучению нашего агента, давайте ** изучим библиотеку и среды, которые мы собираемся использовать**.

## Sample Factory (Фабрика образцов)

[Sample Factory](https://www.samplefactory.dev/) является одной из **самых быстрых библиотек RL, ориентированных на очень эффективные синхронные и асинхронные реализации градиентов политики (PPO)**.

Sample Factory (Фабрика образцов) тщательно **протестирована, используется многими исследователями и практиками** и активно поддерживается. Известно, что наша реализация **обеспечивает производительность SOTA в различных областях при минимизации времени обучения RL-эксперименту и требований к оборудованию**.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit9/samplefactoryenvs.png" alt="Sample factory" width="70%"/>


### Ключевые функции

- Высокооптимизированный алгоритм [архитектура](https://www.samplefactory.dev/06-architecture/overview/) для максимальной производительности обучения
- [Синхронный и асинхронный](https://www.samplefactory.dev/07-advanced-topics/sync-async/) режимы обучения
- [Последовательный (однопроцессорный) режим](https://www.samplefactory.dev/07-advanced-topics/serial-mode/) для удобства отладки
- Оптимальная производительность как в средах на базе CPU, так и [в средах с ускорением GPU](https://www.samplefactory.dev/09-environment-integrations/isaacgym/)
- Обучение с одним и несколькими агентами, самостоятельная игра, поддержка [обучения нескольким политикам](https://www.samplefactory.dev/07-advanced-topics/multi-policy-training/) одновременно на одном или нескольких графических процессорах
- Обучение на уровне населения ([PBT](https://www.samplefactory.dev/07-advanced-topics/pbt/))
- Дискретные, непрерывные, гибридные пространства действий
- Векторные, основанные на изображениях, словари пространства наблюдений
- Автоматически создает архитектуру модели путем анализа спецификации пространства действий/наблюдения. Поддерживает [архитектуры пользовательских моделей](https://www.samplefactory.dev/03-customization/custom-models/)
- Предназначен для импорта в другие проекты, [пользовательские среды](https://www.samplefactory.dev/03-customization/custom-environments/)
- Детализация [WandB and Tensorboard summaries](https://www.samplefactory.dev/05-monitoring/metrics-reference/), [пользовательские показатели](https://www.samplefactory.dev/05-monitoring/custom-metrics/)
- [HuggingFace 🤗 интеграция](https://www.samplefactory.dev/10-huggingface/huggingface/) (загрузка обученных моделей и метрик на Hub)
- [Множество](https://www.samplefactory.dev/09-environment-integrations/mujoco/) [примеров](https://www.samplefactory.dev/09-environment-integrations/atari/) [окружающих сред](https://www.samplefactory.dev/09-environment-integrations/vizdoom/) [для интеграции](https://www.samplefactory.dev/09-environment-integrations/dmlab/) с настроенными параметрами и обученными моделями

Все вышеуказанные политики доступны на 🤗 hub. Выполните поиск по тегу[sample-factory](https://huggingface.co/models?library=sample-factory&sort=downloads)

### Как работает sample-factory

Sample-factory - это одна из **наиболее высокооптимизированных реализаций RL, доступных сообществу**.

Это работает путем **порождения нескольких процессов, которые запускают работников развертывания, работников вывода и работника-учащегося**.

*Рабочие* **взаимодействуют через общую память, что снижает стоимость связи между процессами**.

*Работники развертывания* взаимодействуют с окружающей средой и отправляют наблюдения *работникам вывода*.

Работники *вывода* запрашивают фиксированную версию политики и **отправляют действия обратно работнику развертывания**.

После *k* шагов работы по развертыванию отправляют траекторию опыта работнику-учащемуся**, которую он использует для обновления сети политик агента**.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit9/samplefactory.png" alt="Sample factory" width="70%"/>

### Модели актер-критик из Sample-factory

Модели актера и критика в Sample Factory состоят из трех компонентов:

- ** Кодировщик ** - Обрабатывает входные наблюдения (изображения, векторы) и преобразует их в вектор. Это та часть модели, которую вы, скорее всего, захотите настроить.
- ** Ядро ** - Интегрирует векторы из одного или нескольких кодеров, может дополнительно включать одно- или много- слойный LSTM/GRU в агента на основе памяти.
- **Декодер** - Использует дополнительные слои к выходным данным ядра модели перед вычислением выходных данных политики и значения.

Библиотека была разработана для автоматической поддержки любых пространств наблюдения и действий. Пользователи могут легко добавлять свои пользовательские модели. Вы можете узнать больше в [документации](https://www.samplefactory.dev/03-customization/custom-models/#actor-critic-models-in-sample-factory).

## ViZDoom

[ViZDoom](https://vizdoom.cs.put.edu.pl/) это **интерфейс python с открытым исходным кодом для движка Doom Engine**.

Библиотека была создана в 2016 году Мареком Видмухом и Михалом Кемпкой в Институте вычислительной техники Познанского технологического университета, Польша. 

Библиотека позволяет **обучать агентов непосредственно с пикселей экрана в ряде сценариев**, включая командный смертельный поединок, показанный на видео ниже. Поскольку среда ViZDoom основана на игре, созданной в 90-х годах, ее можно запускать на современном оборудовании с ускоренными скоростями**, что позволяет нам довольно быстро осваивать сложное поведение искусственного интеллекта**.

Библиотека включает в себя такие функции, как:

- Мультиплатформенность (Linux, macOS, Windows),
- API для Python и C++,
- [OpenAI Gym](https://www.gymlibrary.dev/) оболочки среды
- Простые в создании пользовательские сценарии (доступны визуальные редакторы, язык сценариев и примеры),
- Асинхронный и синхронизированный однопользовательский и многопользовательский режимы,
- Легкий (несколько Мбайт) и быстрый (до 7000 кадров в секунду в режиме синхронизации, однопоточный),
- Настраиваемое разрешение и параметры рендеринга,
- Доступ к буферу глубины (3D vision),
- Автоматическая маркировка игровых объектов, видимых в кадре,
- Доступ к аудиобуферу
- Доступ к списку действующих лиц/объектов и геометрии карты,
- Закадровый рендеринг и запись эпизода,
- Масштабирование времени в асинхронном режиме.

## Сначала нам нужно установить некоторые зависимости, которые требуются для среды ViZDoom

Теперь, когда наша среда выполнения Colab настроена, мы можем начать с установки зависимостей, необходимых для запуска ViZDoom в Linux.

Для Mac, вам нужно будет следовать инструкциям по установке на [странице github](https://github.com/Farama-Foundation/ViZDoom/blob/master/doc/Quickstart.md#-quickstart-for-macos-and-anaconda3-python-36).

In [ ]:
%%capture
%%bash
# Установите ViZDoom
# https://github.com/mwydmuch/ViZDoom/blob/master/doc/Building.md#-linux

apt-get install build-essential zlib1g-dev libsdl2-dev libjpeg-dev \
nasm tar libbz2-dev libgtk2.0-dev cmake git libfluidsynth-dev libgme-dev \
libopenal-dev timidity libwildmidi-dev unzip ffmpeg

# Boost libraries
apt-get install libboost-all-dev

# Lua binding dependencies
apt-get install liblua5.1-dev

## Затем мы можем установить  Sample Factory и ViZDoom
- - Это может занять 7 минут

In [ ]:
# install python libraries
# thanks toinsson
!pip install sample-factory==2.0.2
!pip install faster-fifo==1.4.2
!pip install vizdoom

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 KB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.5/86.5 KB 9.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.3/831.3 KB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 KB 50.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 20.1 MB/s 

## Настройка среды Doom в sample-factory

In [ ]:
import functools

from sample_factory.algo.utils.context import global_model_factory
from sample_factory.cfg.arguments import parse_full_cfg, parse_sf_args
from sample_factory.envs.env_utils import register_env
from sample_factory.train import run_rl

from sf_examples.vizdoom.doom.doom_model import make_vizdoom_encoder
from sf_examples.vizdoom.doom.doom_params import add_doom_env_args, doom_override_defaults
from sf_examples.vizdoom.doom.doom_utils import DOOM_ENVS, make_doom_env_from_spec


# Registers all the ViZDoom environments
def register_vizdoom_envs():
    for env_spec in DOOM_ENVS:
        make_env_func = functools.partial(make_doom_env_from_spec, env_spec)
        register_env(env_spec.name, make_env_func)

# Sample Factory allows the registration of a custom Neural Network architecture
# See https://github.com/alex-petrenko/sample-factory/blob/master/sf_examples/vizdoom/doom/doom_model.py for more details
def register_vizdoom_models():
    global_model_factory().register_encoder_factory(make_vizdoom_encoder)


def register_vizdoom_components():
    register_vizdoom_envs()
    register_vizdoom_models()

# parse the command line args and create a config
def parse_vizdoom_cfg(argv=None, evaluation=False):
    parser, _ = parse_sf_args(argv=argv, evaluation=evaluation)
    # parameters specific to Doom envs
    add_doom_env_args(parser)
    # override Doom default values for algo parameters
    doom_override_defaults(parser)
    # second parsing pass yields the final configuration
    final_cfg = parse_full_cfg(parser, argv)
    return final_cfg

Теперь, когда настройка завершена, мы можем обучить агента. Мы выбрали здесь, чтобы изучить задание ViZDoom под названием "Высший сбор здоровья".

### Сценарий: Высший сбор здоровья

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit9/Health-Gathering-Supreme.png" alt="Health-Gathering-Supreme" width="70%"/>



Цель этого сценария состоит в том, чтобы ** научить агента выживать, не зная, что заставляет его выживать **. Агент знает только, что ** жизнь драгоценна**, а смерть плоха, поэтому ** он должен узнать, что продлевает его существование и что с этим связано его здоровье **.

Карта представляет собой прямоугольник со стенами и зеленым кислотным полом, который ** периодически причиняет игроку боль**. Изначально есть несколько аптечек, равномерно распределенных по карте. Время от времени с небес падает новая аптечка. ** Аптечки восстанавливают некоторые части здоровья игрока ** - чтобы выжить, агенту необходимо забрать их. Эпизод заканчивается после смерти игрока или по истечении тайм-аута.

Дальнейшая конфигурация:
- Living_reward = 1
- 3 доступные кнопки: повернуть налево, повернуть направо, двигаться вперед
- 1 доступная игровая переменная: ЗДОРОВЬЕ
- смертная казнь = 100

Вы можете узнать больше о сценариях, доступных в ViZDoom [здесь](https://github.com/Farama-Foundation/ViZDoom/tree/master/scenarios). 

Существует также ряд более сложных сценариев, которые были созданы для ViZDoom, таких как те, которые подробно описаны на [этой странице github](https://github.com/edbeeching/3d_control_deep_rl).

## Обучение агента
- - Мы собираемся обучить агента 4000000 шагам, это займет примерно 20 минут

In [ ]:
## Начните тренировку, это должно занять около 15 минут
register_vizdoom_components()

# Сценарий, по которому мы тренируемся сегодня, - это сбор здоровья
# другие сценарии включают "doom_basic", "doom_two_colors_easy", "doom_dm", "doom_dwango5", "doom_my_way_home", "doom_deadly_corridor", "doom_defend_the_center", "doom_defend_the_line"
env = "doom_health_gathering_supreme"
cfg = parse_vizdoom_cfg(argv=[f"--env={env}", "--num_workers=8", "--num_envs_per_worker=4", "--train_for_env_steps=4000000"])

status = run_rl(cfg)

## Давайте посмотрим на эффективность обученной политики и выведем видео агента.

In [ ]:
from sample_factory.enjoy import enjoy
cfg = parse_vizdoom_cfg(argv=[f"--env={env}", "--num_workers=1", "--save_video", "--no_render", "--max_num_episodes=10"], evaluation=True)
status = enjoy(cfg)

## Теперь давайте визуализируем производительность агента

In [ ]:
from base64 import b64encode
from IPython.display import HTML

mp4 = open('/content/train_dir/default_experiment/replay.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=640 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

Агент кое-чему научился, но его производительность могла бы быть лучше. Нам явно нужно было бы тренироваться дольше. Но давайте загрузим эту модель в Хаб.

## Теперь давайте загрузим вашу контрольную точку и видео в Hugging Face Hub

In [ ]:
from huggingface_hub import notebook_login
notebook_login()
!git config --global credential.helper store

In [ ]:
from sample_factory.enjoy import enjoy

hf_username = "ThomasSimonini" # insert your HuggingFace username here

cfg = parse_vizdoom_cfg(argv=[f"--env={env}", "--num_workers=1", "--save_video", "--no_render", "--max_num_episodes=10", "--max_num_frames=100000", "--push_to_hub", f"--hf_repository={hf_username}/rl_course_vizdoom_health_gathering_supreme"], evaluation=True)
status = enjoy(cfg)

## Давайте загрузим другую модель

Работа этого агента была хорошей, но он может добиться большего! Давайте загрузим и визуализируем агента, обученного 10B временным шагам, из центра.

In [ ]:
#загрузка агента с Hub
!python -m sample_factory.huggingface.load_from_hub -r edbeeching/doom_health_gathering_supreme_2222 -d ./train_dir

In [ ]:
!ls train_dir/doom_health_gathering_supreme_2222

In [ ]:
env = "doom_health_gathering_supreme"
cfg = parse_vizdoom_cfg(argv=[f"--env={env}", "--num_workers=1", "--save_video", "--no_render", "--max_num_episodes=10", "--experiment=doom_health_gathering_supreme_2222", "--train_dir=train_dir"], evaluation=True)
status = enjoy(cfg)

In [ ]:
mp4 = open('/content/train_dir/doom_health_gathering_supreme_2222/replay.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=640 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

## Некоторые дополнительные испытания 🏆 : Doom Deathmatch

Обучение агента игре Doom deathmatch ** занимает много часов на более мощной машине, чем та что доступна в Colab **.

К счастью, мы ** уже обучили агента этому сценарию, и он доступен в 🤗 Hub!** Давайте загрузим модель и визуализируем производительность агента.

In [ ]:
# Download the agent from the hub
!python -m sample_factory.huggingface.load_from_hub -r edbeeching/doom_deathmatch_bots_2222 -d ./train_dir

Учитывая, что агент воспроизводится в течение длительного времени, генерация видео может занять ** 10 минут **.

In [ ]:

from sample_factory.enjoy import enjoy
register_vizdoom_components()
env = "doom_deathmatch_bots"
cfg = parse_vizdoom_cfg(argv=[f"--env={env}", "--num_workers=1", "--save_video", "--no_render", "--max_num_episodes=1", "--experiment=doom_deathmatch_bots_2222", "--train_dir=train_dir"], evaluation=True)
status = enjoy(cfg)
mp4 = open('/content/train_dir/doom_deathmatch_bots_2222/replay.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=640 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

Вы ** можете попробовать обучить своего агента в этой среде **, используя приведенный выше код, но не на colab.
**Удачи 🤞 **

Если вы предпочитаете более простой сценарий, **почему бы не попробовать тренироваться в другом сценарии ViZDoom, таком как `doom_deadly_corridor` или `doom_defend_the_center`.**



---


На этом завершается последний блок. Но мы еще не закончили! 🤗 Следующий **бонусный раздел включает в себя некоторые из наиболее интересных, продвинутых и ультрасовременных работ в области глубокого обучения с подкреплением**.

## Keep learning, stay awesome 🤗